In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
# import notebook_helper
import sasc.viz
import scipy.special
import pandas as pd
from sasc.config import RESULTS_DIR
timings_module = __import__('03_timings')
setting = "qa"
EXPT_NAMES = [
    k
    for k in os.listdir(join(RESULTS_DIR, "stories", setting))
    # if "uts03" in k or "uts01" in k
]

In [ ]:
df

In [ ]:
# import neuro.features.stim_utils
# import neuro.data.story_names
# wordseqs = neuro.features.stim_utils.load_story_wordseqs_huge(
#     stories=neuro.data.story_names.get_story_names(all=True))
# diffs = [
#     np.diff(w.data_times)
#     for w in wordseqs.values()
# ]
# diffs = np.concatenate(diffs)
# # diffs = diffs[diffs < 2]
# plt.hist(diffs, cumulative=True, density=True)
# print('mean', np.mean(diffs), 'min', np.min(diffs), 'max', np.max(diffs))
# # mean 0.3740945421461178

# Process timings

In [ ]:
n_skipped = 0
n_processed = 0
for EXPT_NAME in EXPT_NAMES:
    EXPT_DIR = join(RESULTS_DIR, 'stories', setting, EXPT_NAME)
    timings_file = join(EXPT_DIR, 'timings.csv')
    if not os.path.exists(timings_file):
        print('skipping', EXPT_NAME)
        n_skipped += 1
        continue
    # if os.path.exists(join(EXPT_DIR, 'timings_processed.csv')):
        # print('already processed', EXPT_NAME)
        # n_processed += 1

    print('processing', EXPT_NAME)

    df = pd.read_csv(timings_file)
    print('shape', df.shape)
    try:
        df = timings_module.process_timings(df)
        df.to_csv(join(EXPT_DIR, 'timings_processed.csv'), index=False)
        n_processed += 1
        print('\tmean', df['timing'].mean(), 'min',
              df['timing'].min(), 'max', df['timing'].max())
        print('\ttotal time', df['time_running'].max()/60, 'min')
    except Exception as e:
        print('error', e)
        print('skipping', EXPT_NAME)
        n_skipped += 1
        continue
print('n_skipped', n_skipped, 'n_processed', n_processed)

# Visualize

In [ ]:
df = pd.read_csv(join(RESULTS_DIR, 'stories', 'default',
                 'uts03___jun14___seed=3', 'timings_processed.csv'))

# Plotting legend
df['ends_in'] = 'Normal'
df.loc[df['ends_in_period'], 'ends_in'] = 'Period'
df.loc[df['ends_in_comma'], 'ends_in'] = 'Comma'
# df = df.sort_values(by='ends_in')

# Make plot
sns.lmplot(data=df, x='word_len', y='timing', hue='ends_in',
           markers='.', legend=False, aspect=1.5)
plt.xlabel('Word length (characters)')
plt.ylabel('Word timing (sec)')
plt.axhline(np.mean(df['timing']), color='gray')
plt.text(16, np.mean(df['timing']) - 0.02,
         f'Mean: {np.mean(df["timing"]):.2f} sec', color='gray')
# change legend label value
handles, labels = plt.gca().get_legend_handles_labels()
plt.gca().legend(bbox_to_anchor=(1, 0.7), labelcolor='linecolor')
# plt.gca().legend(handles=handles, labels=['Does not end in period', 'Ends in period'], bbox_to_anchor=(1, 0.7))
plt.show()

# Remove repeated words
(This was run preceding pilot3 on the timings)

In [ ]:
# for s in os.listdir('.'):
#     if os.path.isdir(s):
#         print(s)
#         for seed in os.listdir(s):
#             fname = os.path.join(s, seed, 'timings_processed.csv')
#             t = pd.read_csv(fname)
#             # remove rows with repeated word
#             t = t[t['word'] != t['word'].shift(1)]
#             t['time_running'] = np.cumsum(t["timing"])
#             t.to_csv(fname, index=False)